In [1]:
!pip install scikit-image
!pip install keras

In [10]:
import pandas as pd
import numpy as np
import keras
import matplotlib.pyplot as plt

from skimage import feature

%matplotlib inline

### HOG + Baseline model with augmentation

In [3]:
def load_data(dataset: str, channels: int = 1, augmentation: bool = True):

    df = pd.read_csv(dataset)

    df['category'] = df['category'].str.strip()

    train = df.loc[df.category == 'Training',:]
    valid = df.loc[df.category == 'PublicTest',:]
    test  = df.loc[df.category == 'PrivateTest',:]

    del train['Unnamed: 0']
    del valid['Unnamed: 0']
    del test['Unnamed: 0']

    X_train = np.array(train.iloc[:, 1:2305])
    y_train = np.array(train.loc[:, ['y']])

    X_val = np.array(valid.iloc[:, 1:2305])
    y_val = np.array(valid.loc[:, ['y']])

    X_test = np.array(test.iloc[:, 1:2305])
    y_test = np.array(test.loc[:, ['y']])

    del train
    del valid
    del test

    y_train = keras.utils.to_categorical(y_train, 7)
    y_val = keras.utils.to_categorical(y_val, 7)
    y_test = keras.utils.to_categorical(y_test, 7)

    X_train_r = X_train.reshape((len(X_train), 48, 48))
    X_val_r   = X_val.reshape((len(X_val), 48, 48))
    X_test_r   = X_test.reshape((len(X_test), 48, 48))

    X_train_r = X_train_r / 255
    X_val_r = X_val_r / 255
    X_test_r = X_test_r / 255

    if channels == 1:
        X_train_bw = X_train_r.reshape((len(X_train_r), 48, 48, 1))
        X_val_bw = X_val_r.reshape((len(X_val_r), 48, 48, 1))
        X_test_bw = X_test_r.reshape((len(X_test_r), 48, 48, 1))
        return (X_train_bw, X_val_bw, X_test_bw, y_train, y_val, y_test)
    elif channels == 3:
        X_train_rgb = np.stack((X_train_r,) * 3, axis = -1)
        X_val_rgb = np.stack((X_val_r, ) * 3, axis = -1)
        X_test_rgb = np.stack((X_test_r, ) * 3, axis = -1)
        return (X_train_rgb, X_val_rgb, X_test_rgb, y_train, y_val, y_test)
    else:
        return None

In [4]:
X_train_bw, X_val_bw, X_test_bw, y_train, y_val, y_test = load_data('emotions.csv') 

In [31]:
hog_train = np.zeros((len(X_train_bw), 1296))

for i in range(len(X_train_bw)):
    hog_train[i] = feature.hog(X_train_bw[i].reshape((48,48)), block_norm='L2-Hys')

In [32]:
hog_val = np.zeros((len(X_val_bw), 1296))

for i in range(len(X_val_bw)):
    hog_val[i] = feature.hog(X_val_bw[i].reshape((48,48)), block_norm='L2-Hys')

In [38]:
hog_test = np.zeros((len(X_test_bw), 1296))

for i in range(len(X_test_bw)):
    hog_test[i] = feature.hog(X_test_bw[i].reshape((48,48)), block_norm='L2-Hys')

## Build model

In [39]:
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Concatenate
from keras.layers.normalization import BatchNormalization

from keras.models import Model

from keras.metrics import categorical_accuracy

In [44]:
hog_input = keras.layers.Input(shape=(1296,))

image_input = keras.layers.Input(shape=(48, 48, 1))

conv1 = Conv2D(64,(3,3), padding='same')(image_input)
batch1 = BatchNormalization()(conv1)
activ1 = Activation('relu')(batch1)
maxpo1 = MaxPooling2D(pool_size=(2, 2))(activ1)
dropo1 = Dropout(0.5)(maxpo1)

conv2 = Conv2D(128,(5,5), padding='same')(dropo1)
batch2 = BatchNormalization()(conv2)
activ2 = Activation('relu')(batch2)
maxpo2 = MaxPooling2D(pool_size=(2, 2))(activ2)
dropo2 = Dropout(0.5)(maxpo2)

conv3 = Conv2D(512,(3,3), padding='same')(dropo2)
batch3 = BatchNormalization()(conv3)
activ3 = Activation('relu')(batch3)
maxpo3 = MaxPooling2D(pool_size=(2, 2))(activ3)
dropo3 = Dropout(0.5)(maxpo3)

conv4 = Conv2D(512,(3,3), padding='same')(dropo3)
batch4 = BatchNormalization()(conv4)
activ4 = Activation('relu')(batch4)
maxpo4 = MaxPooling2D(pool_size=(2, 2))(activ4)
dropo4 = Dropout(0.5)(maxpo4)

flatt4 = Flatten()(dropo4)

# hog_dense = Dense(1296)(hog_input)

dense1 = Concatenate()([hog_input, flatt4])

dense2 = Dense(256)(dense1)
batch5 = BatchNormalization()(dense2)
activ5 = Activation('relu')(batch5)
dropo5 = Dropout(0.5)(activ5)

dense3 = Dense(512)(dropo5)
batch6 = BatchNormalization()(dense3)
activ6 = Activation('relu')(batch6)
dropo6 = Dropout(0.5)(activ6)

dense4 = Dense(512)(dropo6)
batch7 = BatchNormalization()(dense4)
activ7 = Activation('relu')(batch7)
dropo7 = Dropout(0.5)(activ7)

out = Dense(7, activation='softmax')(dropo7)

model = keras.models.Model(inputs=[image_input, hog_input], outputs=out)

In [45]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_28 (InputLayer)           (None, 48, 48, 1)    0                                            
__________________________________________________________________________________________________
conv2d_50 (Conv2D)              (None, 48, 48, 64)   640         input_28[0][0]                   
__________________________________________________________________________________________________
batch_normalization_57 (BatchNo (None, 48, 48, 64)   256         conv2d_50[0][0]                  
__________________________________________________________________________________________________
activation_57 (Activation)      (None, 48, 48, 64)   0           batch_normalization_57[0][0]     
__________________________________________________________________________________________________
max_poolin

In [46]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[categorical_accuracy])

In [48]:
history = model.fit([X_train_bw, hog_train], y_train, batch_size=128, epochs=100, validation_data=([X_val_bw, hog_val], y_val))

Train on 28709 samples, validate on 3589 samples
Epoch 1/100
28709/28709 [==============================] - 31s 1ms/step - loss: 1.2015 - categorical_accuracy: 0.5414 - val_loss: 1.2018 - val_categorical_accuracy: 0.5447
Epoch 2/100
28709/28709 [==============================] - 31s 1ms/step - loss: 1.1853 - categorical_accuracy: 0.5495 - val_loss: 1.3540 - val_categorical_accuracy: 0.4700
Epoch 3/100
28709/28709 [==============================] - 31s 1ms/step - loss: 1.1595 - categorical_accuracy: 0.5631 - val_loss: 1.3280 - val_categorical_accuracy: 0.4737
Epoch 4/100
28709/28709 [==============================] - 31s 1ms/step - loss: 1.1389 - categorical_accuracy: 0.5706 - val_loss: 1.1714 - val_categorical_accuracy: 0.5511
Epoch 5/100
28709/28709 [==============================] - 31s 1ms/step - loss: 1.1315 - categorical_accuracy: 0.5686 - val_loss: 1.2014 - val_categorical_accuracy: 0.5436
Epoch 6/100
28709/28709 [==============================] - 31s 1ms/step - loss: 1.1042 - ca

28709/28709 [==============================] - 31s 1ms/step - loss: 0.3377 - categorical_accuracy: 0.8780 - val_loss: 1.3802 - val_categorical_accuracy: 0.6038
Epoch 96/100
28709/28709 [==============================] - 30s 1ms/step - loss: 0.3414 - categorical_accuracy: 0.8784 - val_loss: 1.3387 - val_categorical_accuracy: 0.6127
Epoch 97/100
28709/28709 [==============================] - 30s 1ms/step - loss: 0.3384 - categorical_accuracy: 0.8780 - val_loss: 1.2824 - val_categorical_accuracy: 0.5946
Epoch 98/100
28709/28709 [==============================] - 31s 1ms/step - loss: 0.3357 - categorical_accuracy: 0.8806 - val_loss: 1.3699 - val_categorical_accuracy: 0.5988
Epoch 99/100
28709/28709 [==============================] - 30s 1ms/step - loss: 0.3411 - categorical_accuracy: 0.8781 - val_loss: 1.3574 - val_categorical_accuracy: 0.6004
Epoch 100/100
28709/28709 [==============================] - 30s 1ms/step - loss: 0.3348 - categorical_accuracy: 0.8804 - val_loss: 1.3192 - val_cat

In [49]:
model.evaluate([X_test_bw, hog_test], y_test)

3589/3589 [==============================] - 2s 482us/step


[1.3224164700235752, 0.5962663694705496]